In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

# Load the dataset from Hugging Face if it's your first time using it

# dataset = fouh.load_from_hub(
# "Voxel51/Coursera_lecture_dataset_train", 
# dataset_name="lecture_dataset_train", 
# persistent=True)

In [ ]:
#because I have the dataset saved locally, I will load it like so
cloned_dataset = fo.load_dataset("lecture_dataset_train_clone")

In [ ]:
#  #clone the dataset to avoid modifying the original dataset
# cloned_dataset = dataset.clone(name="lecture_dataset_train_clone")

In [ ]:
import fiftyone as fo
import cv2
import numpy as np
from scipy.ndimage import label
from scipy.spatial.distance import pdist, squareform

### Image complexity

We can use Canny edge detection to measure the ratio of edge pixels to total pixels

This metric can be useful because:

1. It provides a measure of the level of detail and intricacy in an image.

2. Higher complexity can indicate more challenging images for object detection.

3. It can help identify images that might require more processing power or sophisticated algorithms for accurate analysis.

4. Understanding image complexity can aid in balancing datasets and evaluating model performance across different complexity levels.


#### Limitations

- **Oversimplification:** Edge detection reduces an image to binary information (edge or non-edge), discarding valuable texture and color information that could be crucial for object detection.

- **Sensitivity to Noise:** Canny edge detection can be sensitive to image noise, potentially leading to inaccurate complexity assessments in noisy images.

- **Parameter Dependency:** The effectiveness of Canny edge detection heavily relies on the chosen threshold parameters (100 and 200 in this case), which may not be optimal for all images in a diverse dataset.

In [ ]:
def calculate_image_complexity(dataset):
    """
    Calculate the complexity of images in a FiftyOne dataset using Canny edge detection and color information.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    None. It just adds the field to the dataset
    """
    for sample in dataset.iter_samples():
        img = cv2.imread(sample.filepath)
        # Convert the image to float32
        img_float = img.astype(np.float32) / 255.0
        # Calculate the color variance for the image
        color_variance = np.var(img_float, axis=(0, 1)).sum()
        # Convert to grayscale for edge detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        edge_complexity = np.sum(edges > 0) / (img.shape[0] * img.shape[1])
        # Combine edge complexity and color variance
        complexity = edge_complexity + color_variance
        sample["image_complexity_score"] = complexity
        sample.save()

In [ ]:
calculate_image_complexity(cloned_dataset)

In [ ]:
fo.launch_app(cloned_dataset)

### Visual clutter 

Calculates the variance of pixel intensities in the image.

This metric is useful because:

1. It measures the level of disorder or chaos in an image, which can impact object detection.

2. High visual clutter can make it more difficult to isolate and identify individual objects.

3. It provides insight into the visual complexity of scenes beyond just object count or density.

4. Understanding visual clutter can help in developing strategies to improve model performance on visually complex images.

In [40]:
def calculate_visual_clutter(dataset):
    """
    Calculate the normalized visual clutter of images in a FiftyOne dataset using pixel intensity variance and color variance.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    None. It just adds the normalized clutter score field to the dataset.
    """
    max_gray_variance = 255 ** 2  # Maximum possible variance for an 8-bit grayscale image
    max_color_variance = 3 * (1.0 ** 2)  # Maximum possible variance for colors in [0, 1]
    max_combined_variance = max_gray_variance + max_color_variance

    for sample in dataset.iter_samples():
        img = cv2.imread(sample.filepath)
        
        # Calculate grayscale variance
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_clutter = np.var(gray)
        
        # Calculate color variance
        img_float = img.astype(np.float32) / 255.0  # Convert to float32
        color_variance = np.var(img_float, axis=(0, 1)).sum()
        
        # Combine both measures
        clutter = gray_clutter + color_variance
        
        # Normalize the clutter score
        normalized_clutter = clutter / max_combined_variance
        
        # Ensure the score is between 0 and 1
        normalized_clutter = np.clip(normalized_clutter, 0, 1)
        
        sample["normalized_clutter_score"] = float(normalized_clutter)
        sample.save()

In [41]:
calculate_visual_clutter(cloned_dataset)

In [ ]:
fo.launch_app(cloned_dataset)

### Object Clutter

An object vlutter score will identify number of detections per image. This is a simple and useful metric. It provides a quick measure of how busy or crowded an image is in terms of objects.

**Pros:**
- Easy to calculate and interpret
- Gives a clear indication of image complexity

**Cons:**
- Doesn't account for object size or distribution
- May not distinguish between genuinely cluttered scenes and scenes with many small objects

**Usefulness:** High, especially as a basic measure of image complexity.

In [ ]:
def object_clutter_score(dataset):
    """
    Calculate the clutter score based on the number of detections per image.

    This metric is useful because:
    1. It provides a simple measure of scene complexity in terms of object count.
    2. Higher clutter scores can indicate more challenging images for object detection.
    3. It helps identify images that may require more processing time or have higher chances of false positives/negatives.
    4. Understanding clutter can aid in balancing datasets and evaluating model performance across different complexity levels.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    dict: A dictionary with image IDs as keys and clutter scores as values.
    """
    clutter_scores = {}

    for sample in dataset:
        num_detections = len(sample.detections)
        clutter_scores[sample.id] = num_detections

    return clutter_scores

### Object diversity

This will measure the number of distinct classes per image. This is an excellent metric for measuring the semantic diversity of an image.

**Pros:**
- Directly measures the variety of object types in an image
- Easy to calculate and interpret

**Cons:**
- Doesn't account for the number of instances of each class
- Treats all classes equally, regardless of their visual or semantic similarity

**Usefulness:** High, particularly for understanding the range of objects a model needs to handle.

3. Objectness score: % of pixels that belong to classes across the whole image

This is a valuable metric for understanding how much of the image is occupied by objects of interest.

**Pros:**
- Provides insight into the density of annotated objects
- Can help identify images with large background areas

**Cons:**
- Doesn't account for the number or diversity of objects
- May be biased towards images with large objects

**Usefulness:** High, especially when combined with other metrics.

In [ ]:
def object_diversity_score(dataset):
    """
    Calculate the instance diversity based on the number of distinct classes per image.

    This metric is useful because:
    1. It quantifies the variety of object types present in an image.
    2. Higher diversity can indicate more complex scenes that require broader object recognition capabilities.
    3. It helps in assessing the range of objects a model needs to handle within a single image.
    4. Understanding instance diversity can guide dataset curation to ensure a wide range of object combinations are represented.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    dict: A dictionary with image IDs as keys and instance diversity scores as values.
    """
    diversity_scores = {}

    for sample in dataset:
        unique_classes = set(det.label for det in sample.detections)
        diversity_scores[sample.id] = len(unique_classes)

    return diversity_scores

### Objectness 

In [ ]:
def calculate_objectness_score(dataset):
    """
    Calculate the objectness score as the percentage of pixels belonging to objects.

    This metric is useful because:
    1. It provides insight into how much of the image is occupied by objects of interest.
    2. Lower scores might indicate images with large background areas or small objects, which can be challenging for detection.
    3. It can help identify images where objects occupy a significant portion of the scene, potentially affecting detection strategies.
    4. Understanding objectness can aid in analyzing model performance relative to object size and prominence in the image.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    dict: A dictionary with image IDs as keys and objectness scores as values.
    """
    objectness_scores = {}

    for sample in dataset:
        total_pixels = sample.metadata.width * sample.metadata.height
        object_pixels = sum(det.bounding_box[2] * det.bounding_box[3] * total_pixels
                            for det in sample.detections)
        objectness_scores[sample.id] = object_pixels / total_pixels

    return objectness_scores

###  Diversity Ratio

This metric considers the number of detections and number of classes per image

This is a more nuanced approach to measuring diversity that takes into account both the number of objects and the variety of classes.

**Pros:**
- Combines quantity and variety of objects
- Can distinguish between images with many objects of few classes and those with fewer objects but more classes

**Cons:**
- May require careful design to balance the influence of object count and class count
- Interpretation might be less intuitive than simpler metrics

**Usefulness:** High, as it provides a more comprehensive view of image complexity.

In [ ]:
def calculate_diversity_ratio(dataset):
    """
    Calculate the diversity ratio considering both number of detections and classes.

    This metric is useful because:
    1. It balances the number of objects with the variety of object types, providing a more nuanced view of image complexity.
    2. It can distinguish between images with many objects of few classes and those with fewer objects but more diverse classes.
    3. Higher ratios might indicate images that require broader object recognition capabilities.
    4. This metric can help in creating balanced datasets that challenge models in different ways.

    Parameters:
    dataset (fiftyone.core.dataset.Dataset): FiftyOne dataset object.

    Returns:
    dict: A dictionary with image IDs as keys and diversity ratio scores as values.
    """
    diversity_ratios = {}

    for sample in dataset:
        num_detections = len(sample.detections)
        num_classes = len(set(det.label for det in sample.detections))
        if num_detections > 0:
            diversity_ratios[sample.id] = num_classes / np.log(num_detections + 1)
        else:
            diversity_ratios[sample.id] = 0

    return diversity_ratios